# Mapping Russian invasion of Ukraine
This sample notebook visualizes the armed conflict events of a specific day.
You need a RapidAPI account and a valid API key.

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureSet
from arcgis.mapping import WebMap
from arcgis.mapping.renderer import generate_simple, generate_classbreaks
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.conflicts import aggregate, query
from georapid.formats import OutFormat

## Define the date of interest

In [ ]:
date_of_interest = datetime(2022, 2, 24)

## Utility functions for mapping capabilities

In [ ]:
def create_osm_webmap():
    """Creates a simple web map using the OSM basemap."""
    webmap = WebMap()
    webmap.basemap = 'osm'
    return webmap

def create_events_renderer():
    """Creates a simple renderer representing events as areas of interests."""
    renderer = generate_simple(geometry_type='Polygon', colors=[171, 31, 42, 191])
    renderer_outline = {
        'type': 'esriSLS',
        'color': [0, 0, 0, 191],
        'width': 0.75,
        'style': 'esriSLSSolid'
    }
    renderer['symbol']['outline'] = renderer_outline
    return renderer

def create_aggregated_renderer(spatial_df, field = 'count'):
    """Create a class-breaks renderer representing aggregated events as hexagonal spatial bins."""
    renderer = generate_classbreaks(sdf_or_series=spatial_df, geometry='Polygon', colors='OrRd', field=field)
    return renderer

## Connect to RapidAPI
The factory implementation expects a system environment variable named 'x_rapid_api' containing your API key.

In [ ]:
host = 'geoconflicts.p.rapidapi.com'
conflicts_client = EnvironmentClientFactory.create_client_with_host(host)

## Utility functions for intelligence capabilities

In [ ]:
def create_aggregated_map(conflicts_client, date_of_interest):
    """Creates a map of the aggregated armed conflict events."""
    focus_map = create_osm_webmap()
    aggregated_features_dict = aggregate(conflicts_client, date=date_of_interest, format=OutFormat.ESRI)
    aggregated_events_featureset = FeatureSet.from_dict(aggregated_features_dict)
    aggregated_events_renderer = create_aggregated_renderer(aggregated_events_featureset.sdf)
    focus_map.add_layer(aggregated_events_featureset, {'renderer': aggregated_events_renderer, 'opacity': 0.7})
    return focus_map

def create_events_map(conflicts_client, date_of_interest):
    """Creates a map of the armed conflict events."""
    focus_map = create_osm_webmap()
    features_dict = query(conflicts_client, date=date_of_interest, format=OutFormat.ESRI)
    events_featureset = FeatureSet.from_dict(features_dict)
    events_renderer = create_events_renderer()
    focus_map.add_layer(events_featureset, {'renderer': events_renderer, 'opacity': 0.7})
    return focus_map

def aggregate_admin_regions(conflicts_client, date_of_interest):
    """Aggregates the armed conflict events by first admin level and shows the top five."""
    features_dict = query(conflicts_client, date=date_of_interest, format=OutFormat.ESRI)
    events_featureset = FeatureSet.from_dict(features_dict)
    return events_featureset.sdf.groupby(by='admin1').count().sort_values('name', ascending=False)[:5]

## Mapping the armed conflict events
The first web map shows the aggregated armed conflict events of 24th February 2022.

In [ ]:
# Mapping the aggregated armed conflict events
create_aggregated_map(conflicts_client, date_of_interest)

In [ ]:
# Mapping the armed conflict events
create_events_map(conflicts_client, date_of_interest)

## Show some stats

In [ ]:
aggregate_admin_regions(conflicts_client, date_of_interest)

## Mapping monthly hotspots

In [ ]:
create_aggregated_map(conflicts_client, datetime(2022, 3, 14))

In [ ]:
create_aggregated_map(conflicts_client, datetime(2022, 4, 25))